In [6]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import lightgbm as lgb

class UJIDataset(torch.utils.data.Dataset):
    def __init__(self, root, train = True, transform = None, target_transform = None, download = False):
        self.root = root
        dir_path = self.root + '/UJIndoorLoc'
        zip_path = self.root + '/uji_uil.zip'
        dataset_training_file = dir_path + '/trainingData.csv'
        dataset_validation_file = dir_path + '/validationData.csv'
        # Load independent variables (WAPs values)
        if train:
            dataset_file = dataset_training_file
        else:
            dataset_file = dataset_validation_file
        file = open(dataset_file, 'r')
        # Load independent variables
        file_load = np.loadtxt(file, delimiter = ',', skiprows = 1)
        self.x = file_load[:, 0 : 520]
        self.x = np.concatenate([self.x, file_load[:, 524 : 529]])
        # Load dependent variables
        self.y = file_load[:, 520 : 524]
        file.close()
        # Regularization of independent variables
        self.x[self.x == 100] = np.nan    # WAP not detected
        self.x = self.x + 104             # Convert into positive values
        self.x = self.x / 104             # Regularize into scale between 0 and 1
        # Reduce the number of dependent variables by combining building number and floor into one variable: area
        self.area = self.y[:, 3] * 5 + self.y[:, 2]
    def to_tensor(self):
        self.x = torch.from_numpy(self.x).float()
        self.y = torch.from_numpy(self.y).float()
        self.area = torch.from_numpy(self.area).float()
    # Return the target instance (row)
    def __getitem__(self, index_row):
        return self.x[index_row, :], self.y[index_row, :]
    # Return the number of instances (the number of rows)
    def __len__(self, dim = 0):
        return int(self.x.size()[0])

Import dataset with lightgbm

In [5]:
a = torch.zeros([2,2])
print(a)
b = torch.ones([2,2])
c = torch.cat((a, b), dim = 1)
print(c)

tensor([[0., 0.],
        [0., 0.]])
tensor([[0., 0., 1., 1.],
        [0., 0., 1., 1.]])


In [7]:
# Load training dataset
dataset_train = UJIDataset('./data', train = True)
#gb_train_data = lgb.Dataset(

MemoryError: Unable to allocate array with shape (19937, 1, 529) and data type float64